In [1]:
import pandas as pd

In [2]:
papers = pd.read_excel('policing_papers.xlsx', sheet_name='Papers_cleaned')

In [3]:
papers.head()

,First.Author.Last.First.MI.,Author.Affliation,Paper.Title,Study.Dataset.Name,Grantor,Grant.UID,Paper.DOI,Year.Published,Link.to.Paper,Journal,...,policedepiction,effects,Partners,totalpartnerorgs,modeling,peer_review,rq,rq_type,AC,AC_Type
0,"Gau, Jacinta M.",University of Central Florida,"""Just because Somebody Is in a White Jacket an...",NaN,NIJ,6b04e6bc-3a77-45a6-b164-a4714a178f5e,10.1007/s12103-020-09569-7,2021,https://www-webofscience-com.proxy.uchicago.ed...,American Journal of Criminal Justice: The Jour...,...,applied,Descriptive,NaN,NaN,Qualitative,1,How do police arrest occupational criminals?,Police behavior,0,NaN
1,"O'Neal, Eryn",Sam Houston State University,"""Most [False Reports] Involve Teens"": Officer ...",NaN,NIJ,25a590cd-d37d-489a-9b64-40898c021b08,10.1177/1077801219828537,2020,https://www-webofscience-com.proxy.uchicago.ed...,Violence Against Women,...,negative,Descriptive,NaN,NaN,Descriptive,1,How do police perceive or teenage sexual assau...,Police thought,1,Police opinion as public policy
2,"Donner, Christopher M.",Loyola University Chicago,"""The best predictor of future behavior is ...""...",NaN,NIJ,30789ba0-d541-4b7d-9540-2393160ca592,10.1080/0735648X.2018.1537882,2019,https://www-webofscience-com.proxy.uchicago.ed...,Journal of Crime and Justice,...,negative,Negative,NaN,NaN,Simple Regression,1,Does past misconduct affect officer perception...,Police thought,0,NaN
3,"O'Neal, Eryn",Sam Houston State University,"""Victim is Not Credible"": The Influence of Rap...",NaN,NIJ,25a590cd-d37d-489a-9b64-40898c021b08,10.1080/07418825.2017.1406977,2019,https://www-webofscience-com.proxy.uchicago.ed...,Justice Quarterly,...,negative,Descriptive,NaN,NaN,Descriptive,1,How do police perceive of rape victims?,Police thought,0,NaN
4,"Schuck, Amie M.",University of Illinois Chicago,A Longitudinal Study of the Effects of Academy...,NaN,NIJ,30789ba0-d541-4b7d-9540-2393160ca592,10.1177/10986111211029958,2021,https://www-webofscience-com.proxy.uchicago.ed...,Police Quarterly,...,negative,Negative,NaN,NaN,Simple Regression,1,How does police academy training affect office...,Police training evaluation,0,NaN


In [4]:
papers['AC'].value_counts()

AC
0    113
1     30
Name: count, dtype: int64

In [5]:
papers.groupby('AC')[['peer_review']].agg('sum')

,peer_review
AC,
0,74
1,19


In [6]:
papers.groupby('AC').modeling.value_counts(dropna=False)

AC  modeling         
0   Descriptive          39
    Simple Regression    34
    RCT                  12
    Qualitative          11
    NaN                   8
    Quasi-Experiment      7
    Mixed method          2
1   Descriptive          20
    Simple Regression     6
    Qualitative           2
    RCT                   1
    NaN                   1
Name: count, dtype: int64

In [7]:
papers.groupby('AC').effects.value_counts(dropna=False)

AC  effects                  
0   Descriptive                  44
    Positive                     26
    Negative                     12
    Mixed                        10
    Null                          9
    Applied                       8
    NaN                           4
1   Descriptive                  15
    Positive                      8
    Mixed                         4
    Applied                       2
    Mixed but mostly Positive     1
Name: count, dtype: int64

In [8]:
papers.groupby('AC').Grantor.value_counts(dropna=False)

AC  Grantor  
0   NIJ          72
    Arnold       36
    MacArthur     5
1   NIJ          17
    Arnold       11
    MacArthur     2
Name: count, dtype: int64

In [9]:
altmetric = pd.read_excel('policing_papers.xlsx', sheet_name='AltmetricCitations')

In [10]:
mentions_by_paper = altmetric.pivot_table(index='cited_doi', values='cited_altmetric_id', columns='citation_medium', aggfunc='count').fillna(0).reset_index()
mentions_by_paper['all_news'] = mentions_by_paper.loc[:, ['blogs', 'policy', 'news', 'reddit']].sum(axis=1)

In [11]:
papers['doi_merge'] = papers['Paper.DOI'].str.lower()
mentions_by_paper['doi_merge'] = mentions_by_paper['cited_doi'].str.lower()

In [12]:
papers_with_mentions = papers.merge(mentions_by_paper, on='doi_merge', how='left')

In [13]:
papers_with_mentions['all_news'] = papers_with_mentions.all_news.fillna(0)

In [14]:
peer_reviewed = papers_with_mentions.doi_merge.notna()
post_floyd = papers_with_mentions['Year.Published'] > 2020

In [15]:
papers_with_mentions[peer_reviewed].groupby('AC').all_news.agg(['mean', 'median', 'std', 'max', 'min', 'count'])

,mean,median,std,max,min,count
AC,,,,,,
0,4.608108,0.0,16.152372,108.0,0.0,74
1,3.277778,0.0,10.051664,41.0,0.0,18


In [16]:
papers_with_mentions[peer_reviewed&post_floyd].groupby('AC')['all_news'].agg('sum')

AC
0    29.0
1    56.0
Name: all_news, dtype: float64

In [17]:
papers_with_mentions[papers_with_mentions.doi_merge.notna()].all_news.agg(['mean', 'median', 'std', 'max', 'min', 'count'])

mean        4.347826
median      0.000000
std        15.114533
max       108.000000
min         0.000000
count      92.000000
Name: all_news, dtype: float64